# Homework 4: Conditional GAN for Bond Pricing

**Course**: Deep Learning in Finance (Baruch MFE 2025)  
**Total Points**: 100

---

## Learning Objectives

After completing this homework, you will be able to:

1. **Compare** point predictions from conditional GANs vs traditional regression
2. **Calculate** and interpret calibrated prediction intervals
3. **Generate** correlated market scenarios using GANs
4. **Compute** portfolio risk metrics (VaR, CVaR) from distributional predictions
5. **Analyze** correlation structures and diversification opportunities
6. **Articulate** when to use distributional vs point prediction models

---

## Instructions

- **All model training code is provided** - you do NOT need to implement the GAN
- **Your task**: Complete the analysis questions and provide interpretations
- **Coding**: Some questions require simple calculations (means, percentiles, etc.)
- **Interpretation**: Write 3-5 sentences explaining your findings
- **Run all cells** in order before starting the questions

---

## Grading Rubric

- **Section 1 (Model Comparison)**: 20 points
- **Section 2 (Distributional Predictions)**: 25 points
- **Section 3 (Portfolio Risk Analysis)**: 30 points
- **Section 4 (Correlation & Diversification)**: 25 points

**Total**: 100 points

---

# Part A: Setup & Model Training (Provided - Run All Cells)

The following sections load data, train both ElasticNet and cGAN models.  
**Simply run all cells below** until you reach the homework questions.

## 1. Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print("Imports complete")

In [ ]:
# Set data/output directory
DATA_DIR = Path.cwd()
OUTPUT_DIR = DATA_DIR

print(f"Data directory: {DATA_DIR}")

# Verify file exists
trace_file = DATA_DIR / 'trace_hy_merged.csv'
if not trace_file.exists():
    raise FileNotFoundError(f"trace_hy_merged.csv not found in {DATA_DIR}")

print(f"✓ Input file found")

## 2. Load Data & Train/Test Split

In [ ]:
# Load TRACE data
trace_hy = pd.read_csv(
    DATA_DIR / "trace_hy_merged.csv",
    parse_dates=["date", "maturity", "offering_date"]
)

print("=" * 80)
print("DATA LOADED")
print("=" * 80)
print(f"Records: {len(trace_hy):,}")
print(f"Unique bonds: {trace_hy['cusip_9'].nunique():,}")
print(f"Trading days: {trace_hy['date'].nunique():,}")
print(f"Date range: {trace_hy['date'].min()} to {trace_hy['date'].max()}")

# Chronological train/test split - ONE MONTH EACH
date_range = sorted(trace_hy['date'].unique())
train_size = 22  # ~1 month
test_size = 22   # ~1 month

train_dates = date_range[:train_size]
test_dates = date_range[train_size:train_size + test_size]

print(f"\nTRAIN/TEST SPLIT")
print("=" * 80)
print(f"Train: {train_dates[0].date()} to {train_dates[-1].date()} ({len(train_dates)} days)")
print(f"Test:  {test_dates[0].date()} to {test_dates[-1].date()} ({len(test_dates)} days)")

# Split data
train_data = trace_hy[trace_hy['date'].isin(train_dates)].copy()
test_data = trace_hy[trace_hy['date'].isin(test_dates)].copy()

print(f"\nTrain samples: {len(train_data):,}")
print(f"Test samples: {len(test_data):,}")
print(f"Train bonds: {train_data['cusip_9'].nunique()}")
print(f"Test bonds: {test_data['cusip_9'].nunique()}")
print("=" * 80)

# Universe for scenario analysis
universe_df = trace_hy.drop_duplicates('cusip_9', keep='last').copy()
print(f"\nUniverse size: {len(universe_df)} unique bonds")

## 3. Feature Engineering

In [ ]:
def prepare_features(bonds_df, reference_date, include_lagged_price=False, lagged_prices=None, feature_columns=None):
    """
    Prepare feature matrix for regression.
    """
    df = bonds_df.copy()
    
    # Time-varying features
    df['days_to_maturity'] = (df['maturity'] - reference_date).dt.days
    df['years_to_maturity'] = df['days_to_maturity'] / 365.25
    df['days_since_issue'] = (reference_date - df['offering_date']).dt.days
    df['years_since_issue'] = df['days_since_issue'] / 365.25
    
    # Handle missing values
    if 'amount_outstanding' in df.columns:
        sector_rating_median = df.groupby(['sector', 'rating_broad'])['amount_outstanding'].transform('median')
        df['amount_outstanding'] = df['amount_outstanding'].fillna(sector_rating_median)
        df['amount_outstanding'] = df['amount_outstanding'].fillna(df['amount_outstanding'].median())
    
    if 'offering_yield' in df.columns:
        rating_median = df.groupby('rating_broad')['offering_yield'].transform('median')
        df['offering_yield'] = df['offering_yield'].fillna(rating_median)
        df['offering_yield'] = df['offering_yield'].fillna(df['offering_yield'].median())
    
    # Numerical features
    numerical_features = []
    if 'amount_outstanding' in df.columns:
        numerical_features.append('amount_outstanding')
    if 'offering_yield' in df.columns:
        numerical_features.append('offering_yield')
    numerical_features.extend(['years_to_maturity', 'years_since_issue'])
    
    # Categorical features
    categorical_features = []
    if 'sector' in df.columns:
        categorical_features.append('sector')
    if 'rating_broad' in df.columns:
        categorical_features.append('rating_broad')
    if 'seniority' in df.columns:
        categorical_features.append('seniority')
    if 'coupon_type_label' in df.columns:
        categorical_features.append('coupon_type_label')
    
    # Binary features
    binary_features = []
    for feat in ['putable', 'redeemable', 'convertible', 'rule_144a']:
        if feat in df.columns:
            df[feat] = (df[feat] == 'Y').astype(int)
            binary_features.append(feat)
    
    # Build feature matrix
    X = pd.DataFrame(index=df.index)
    
    for feat in numerical_features:
        X[feat] = df[feat]
    
    for feat in binary_features:
        X[feat] = df[feat]
    
    for feat in categorical_features:
        dummies = pd.get_dummies(df[feat], prefix=feat, drop_first=True)
        X = pd.concat([X, dummies], axis=1)
    
    if include_lagged_price and lagged_prices is not None:
        X['lagged_price'] = df['cusip_9'].map(lagged_prices)
        X['lagged_price'] = X['lagged_price'].fillna(X['lagged_price'].median())
    
    X = X.fillna(0)
    
    if feature_columns is not None:
        for col in feature_columns:
            if col not in X.columns:
                X[col] = 0
        X = X[feature_columns]
    
    return X, list(X.columns)

print("Feature engineering function defined")

## 4. Train ElasticNet Baseline

In [ ]:
print("Training ElasticNet on training period...\n")

# Prepare features for first training day
first_date = train_dates[0]
day1_data = train_data[train_data['date'] == first_date]
X_day1, feature_names = prepare_features(day1_data, first_date, include_lagged_price=False)
y_day1 = day1_data['price'].values

# Train initial model
scaler_enet = StandardScaler()
X_day1_scaled = scaler_enet.fit_transform(X_day1)

model_enet = ElasticNetCV(
    l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99],
    cv=5,
    max_iter=10000,
    random_state=42,
    n_jobs=-1
)
model_enet.fit(X_day1_scaled, y_day1)

print(f"ElasticNet trained on {len(train_data)} samples")
print(f"Features: {len(feature_names)}")
print(f"Best alpha: {model_enet.alpha_:.6f}")
print(f"Best l1_ratio: {model_enet.l1_ratio_:.2f}")

## 5. Define & Train Conditional GAN

In [ ]:
class ConditionalGenerator(nn.Module):
    """
    Generator: G(z, features) -> price
    """
    def __init__(self, noise_dim, condition_dim, hidden_dim=128, output_dim=1):
        super().__init__()
        self.noise_dim = noise_dim
        input_dim = noise_dim + condition_dim
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, output_dim),
        )
    
    def forward(self, z, features):
        gen_input = torch.cat([z, features], dim=1)
        return self.net(gen_input)


class ConditionalDiscriminator(nn.Module):
    """
    Discriminator: D(features, price) -> [0, 1]
    """
    def __init__(self, condition_dim, hidden_dim=128):
        super().__init__()
        input_dim = condition_dim + 1
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, features, price):
        disc_input = torch.cat([features, price], dim=1)
        return self.net(disc_input).view(-1)


print("Conditional GAN architecture defined")

In [ ]:
# Prepare training data
X_train_list = []
y_train_list = []

for train_date in train_dates:
    day_data = train_data[train_data['date'] == train_date]
    if len(day_data) == 0:
        continue
    
    X_day, _ = prepare_features(day_data, train_date, include_lagged_price=False, feature_columns=feature_names)
    y_day = day_data['price'].values
    
    X_train_list.append(X_day.values)
    y_train_list.append(y_day)

X_train_cgan = np.vstack(X_train_list)
y_train_cgan = np.concatenate(y_train_list).reshape(-1, 1)

print(f"Training samples: {len(X_train_cgan):,}")

# Normalize
scaler_cgan = StandardScaler()
X_train_cgan_scaled = scaler_cgan.fit_transform(X_train_cgan)

price_mean = y_train_cgan.mean()
price_std = y_train_cgan.std()
y_train_cgan_normalized = (y_train_cgan - price_mean) / price_std

# Create dataset
dataset = TensorDataset(
    torch.tensor(X_train_cgan_scaled, dtype=torch.float32),
    torch.tensor(y_train_cgan_normalized, dtype=torch.float32)
)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, drop_last=True)

print(f"DataLoader: {len(dataloader)} batches")

In [ ]:
print("Training Conditional GAN...\n")
print("=" * 80)

# Hyperparameters
noise_dim = 20
condition_dim = X_train_cgan_scaled.shape[1]
hidden_dim = 128
n_epochs = 2000
lr = 1e-4

# Initialize models
G = ConditionalGenerator(noise_dim, condition_dim, hidden_dim).to(device)
D = ConditionalDiscriminator(condition_dim, hidden_dim).to(device)

# Optimizers
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss
criterion = nn.BCELoss()

# Training loop
g_losses = []
d_losses = []

print(f"Epochs: {n_epochs}")
print("Training...")
print("=" * 80)

for epoch in range(n_epochs):
    epoch_g_loss = 0
    epoch_d_loss = 0
    n_batches = 0
    
    for features_real, prices_real in dataloader:
        batch_size = features_real.size(0)
        features_real = features_real.to(device)
        prices_real = prices_real.to(device)
        
        # Train Discriminator
        z = torch.randn(batch_size, noise_dim, device=device)
        prices_fake = G(z, features_real).detach()
        
        d_real = D(features_real, prices_real)
        d_fake = D(features_real, prices_fake)
        
        d_loss = criterion(d_real, torch.ones_like(d_real)) + \
                 criterion(d_fake, torch.zeros_like(d_fake))
        
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Train Generator
        z = torch.randn(batch_size, noise_dim, device=device)
        prices_fake = G(z, features_real)
        
        g_loss = criterion(D(features_real, prices_fake), torch.ones_like(D(features_real, prices_fake)))
        
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        epoch_g_loss += g_loss.item()
        epoch_d_loss += d_loss.item()
        n_batches += 1
    
    g_losses.append(epoch_g_loss / n_batches)
    d_losses.append(epoch_d_loss / n_batches)
    
    if (epoch + 1) % 500 == 0:
        print(f"Epoch [{epoch+1:4d}/{n_epochs}] | D_loss: {d_losses[-1]:.4f} | G_loss: {g_losses[-1]:.4f}")

print("\n" + "=" * 80)
print("cGAN TRAINING COMPLETE")
print("=" * 80)

## 6. Generate Test Predictions

In [ ]:
def generate_cgan_samples(G, features, n_samples=1000, noise_dim=20, device=device):
    """
    Generate multiple price samples for given features.
    """
    n_bonds = features.shape[0]
    samples = np.zeros((n_bonds, n_samples))
    
    with torch.no_grad():
        features_tensor = torch.tensor(features, dtype=torch.float32).to(device)
        
        for i in range(n_samples):
            z = torch.randn(n_bonds, noise_dim, device=device)
            prices_normalized = G(z, features_tensor).cpu().numpy().squeeze()
            # Denormalize
            prices = prices_normalized * price_std + price_mean
            samples[:, i] = prices
    
    return samples


print("Generating predictions on test set...\n")

# Storage
test_results = {
    'date': [],
    'cusip': [],
    'actual': [],
    'enet_pred': [],
    'cgan_mean': [],
    'cgan_p05': [],
    'cgan_p25': [],
    'cgan_p50': [],
    'cgan_p75': [],
    'cgan_p95': []
}

# Store all cGAN samples for later analysis
all_cgan_samples = {}  # cusip -> array of 1000 samples

for test_date in test_dates:
    day_data = test_data[test_data['date'] == test_date]
    
    if len(day_data) == 0:
        continue
    
    # Prepare features
    X_day, _ = prepare_features(day_data, test_date, include_lagged_price=False, feature_columns=feature_names)
    X_day_scaled = scaler_cgan.transform(X_day.values)
    y_actual = day_data['price'].values
    cusips = day_data['cusip_9'].values
    
    # ElasticNet predictions
    enet_preds = model_enet.predict(scaler_enet.transform(X_day.values))
    
    # cGAN predictions (1000 samples per bond)
    cgan_samples = generate_cgan_samples(G, X_day_scaled, n_samples=1000, noise_dim=noise_dim, device=device)
    
    # Store results
    for j in range(len(day_data)):
        cusip = cusips[j]
        test_results['date'].append(test_date)
        test_results['cusip'].append(cusip)
        test_results['actual'].append(y_actual[j])
        test_results['enet_pred'].append(enet_preds[j])
        test_results['cgan_mean'].append(cgan_samples[j].mean())
        test_results['cgan_p05'].append(np.percentile(cgan_samples[j], 5))
        test_results['cgan_p25'].append(np.percentile(cgan_samples[j], 25))
        test_results['cgan_p50'].append(np.percentile(cgan_samples[j], 50))
        test_results['cgan_p75'].append(np.percentile(cgan_samples[j], 75))
        test_results['cgan_p95'].append(np.percentile(cgan_samples[j], 95))
        
        # Store samples for this bond (last date only to save memory)
        all_cgan_samples[cusip] = cgan_samples[j]

# Convert to DataFrame
test_df = pd.DataFrame(test_results)

print(f"Test predictions complete: {len(test_df):,} samples")
print(f"Generated 1000 price samples per bond-day")

---

# Part B: Homework Questions (Complete These)

Now that both models are trained and test predictions are generated, complete the following analysis questions.

---

## Section 1: Model Comparison (20 points)

### Question 1.1: Generate Predictions and Calculate Metrics (10 points)

**Task:**
- Extract actual prices and predictions from `test_df`
- Calculate RMSE (Root Mean Squared Error) for both models
- Calculate MAE (Mean Absolute Error) for both models
- Create a scatter plot comparing actual vs predicted prices for both models

**Hints:**
- Use `test_df['actual']`, `test_df['enet_pred']`, `test_df['cgan_mean']`
- RMSE formula: `np.sqrt(mean_squared_error(actual, predicted))`
- MAE formula: `mean_absolute_error(actual, predicted)`

In [ ]:
# TODO: Extract actual prices and predictions
actual = test_df['actual'].values
enet_pred = # TODO: Extract ElasticNet predictions
cgan_pred = # TODO: Extract cGAN mean predictions

# TODO: Calculate metrics
enet_rmse = # TODO: Calculate RMSE for ElasticNet
cgan_rmse = # TODO: Calculate RMSE for cGAN

enet_mae = # TODO: Calculate MAE for ElasticNet
cgan_mae = # TODO: Calculate MAE for cGAN

# Print results
print("=" * 60)
print("POINT PREDICTION COMPARISON")
print("=" * 60)
print(f"ElasticNet - RMSE: {enet_rmse:.4f}, MAE: {enet_mae:.4f}")
print(f"cGAN       - RMSE: {cgan_rmse:.4f}, MAE: {cgan_mae:.4f}")
print("=" * 60)

# TODO: Create scatter plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ElasticNet
axes[0].scatter(# TODO: Add actual vs predicted for ElasticNet
axes[0].plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Price')
axes[0].set_ylabel('Predicted Price')
axes[0].set_title(f'ElasticNet (RMSE: {enet_rmse:.2f})')
axes[0].grid(True, alpha=0.3)

# cGAN
axes[1].scatter(# TODO: Add actual vs predicted for cGAN
axes[1].plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Price')
axes[1].set_ylabel('Predicted Price')
axes[1].set_title(f'cGAN Mean (RMSE: {cgan_rmse:.2f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Question 1.2: Interpretation (10 points)

**Answer the following questions in 3-5 sentences each:**

1. Which model has better point prediction accuracy (lower RMSE/MAE)? By how much?

2. Looking at the scatter plots, do you see any systematic patterns in the errors (e.g., underprediction for high prices, overprediction for low prices)?

3. What advantage does the cGAN offer beyond point predictions that ElasticNet cannot provide?

**YOUR ANSWER HERE:**

1. [Your interpretation of which model is better and by how much]

2. [Your analysis of the scatter plot patterns]

3. [Your explanation of cGAN's advantages]

---

## Section 2: Distributional Predictions (25 points)

### Question 2.1: Prediction Intervals and Coverage (15 points)

**Task:**
- Calculate empirical coverage for the 90% prediction interval (5th to 95th percentile)
- Calculate empirical coverage for the 50% prediction interval (25th to 75th percentile)
- Visualize prediction intervals vs actual prices for 20 randomly selected bonds

**Hints:**
- Coverage = fraction of actual prices that fall within the interval
- For 90% interval: check if `actual >= p05` AND `actual <= p95`

In [ ]:
# TODO: Calculate coverage
within_90 = # TODO: Calculate fraction of actuals within [p05, p95]
within_50 = # TODO: Calculate fraction of actuals within [p25, p75]

print("=" * 60)
print("PREDICTION INTERVAL COVERAGE")
print("=" * 60)
print(f"50% Interval Coverage: {within_50:.2%} (expected: 50%)")
print(f"90% Interval Coverage: {within_90:.2%} (expected: 90%)")
print("=" * 60)

# TODO: Visualize intervals for 20 random bonds
# Select most frequently traded bonds
top_bonds = test_df.groupby('cusip').size().nlargest(20).index
sample_data = test_df[test_df['cusip'].isin(top_bonds)].copy()
sample_data = sample_data.sort_values(['cusip', 'date'])

# Pick one bond to visualize in detail
sample_cusip = top_bonds[0]
bond_data = sample_data[sample_data['cusip'] == sample_cusip]

# TODO: Create time series plot with intervals
fig, ax = plt.subplots(figsize=(14, 6))

# TODO: Plot actual prices
# TODO: Plot cGAN mean
# TODO: Fill between for 90% interval
# TODO: Fill between for 50% interval

ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.set_title(f'Prediction Intervals: {sample_cusip}')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Question 2.2: Interpretation (10 points)

**Answer the following questions in 3-5 sentences each:**

1. What does it mean if the empirical coverage is 92% for a 90% prediction interval? Is this good or bad?

2. Looking at your time series plot, do the actual prices mostly fall within the prediction intervals? Are there any periods where the model is overconfident or underconfident?

3. Why is this capability valuable for risk management? Think about how a trader or risk manager might use these intervals.

**YOUR ANSWER HERE:**

1. [Your interpretation of calibration and coverage]

2. [Your analysis of the time series plot]

3. [Your explanation of practical applications]

---

## Section 3: Portfolio Risk Analysis (30 points)

### Question 3.1: Generate Correlated Scenarios (10 points)

**Task:**
- Generate 1000 correlated market scenarios for all bonds in the universe
- Create an equal-weighted portfolio
- Calculate portfolio value in each scenario
- Calculate 95% VaR and CVaR

**Starter code provided below:**

In [ ]:
# Pick a test date for scenario analysis
analysis_date = test_dates[len(test_dates)//2]

print(f"Generating scenarios for {analysis_date.date()}...")

# Get ALL bonds in universe with features
X_all, _ = prepare_features(universe_df, analysis_date, include_lagged_price=False, feature_columns=feature_names)
X_all_scaled = scaler_cgan.transform(X_all.values)
n_bonds = len(universe_df)

print(f"Universe size: {n_bonds} bonds")

# Generate correlated scenarios
n_scenarios = 1000
scenarios = np.zeros((n_scenarios, n_bonds))

with torch.no_grad():
    features_tensor = torch.tensor(X_all_scaled, dtype=torch.float32).to(device)
    
    for s in range(n_scenarios):
        # KEY: Use same z for all bonds → captures correlations!
        z = torch.randn(n_bonds, noise_dim, device=device)
        prices_normalized = G(z, features_tensor).cpu().numpy().squeeze()
        prices = prices_normalized * price_std + price_mean
        scenarios[s] = prices

print(f"Scenario matrix shape: {scenarios.shape} (scenarios × bonds)")

In [ ]:
# TODO: Create equal-weighted portfolio
weights = # TODO: Create array of equal weights (1/n_bonds for each bond)

# TODO: Calculate portfolio value in each scenario
portfolio_values = # TODO: Matrix multiply scenarios with weights

# TODO: Calculate VaR and CVaR
var_95 = # TODO: Calculate 5th percentile of portfolio values
cvar_95 = # TODO: Calculate mean of values below VaR_95

print("=" * 60)
print("PORTFOLIO RISK METRICS")
print("=" * 60)
print(f"Portfolio Mean: {portfolio_values.mean():.4f}")
print(f"Portfolio Std:  {portfolio_values.std():.4f}")
print(f"VaR (95%):      {var_95:.4f}")
print(f"CVaR (95%):     {cvar_95:.4f}")
print("=" * 60)

# TODO: Create histogram of portfolio values
fig, ax = plt.subplots(figsize=(12, 6))
# TODO: Plot histogram
# TODO: Add vertical line for VaR
# TODO: Add vertical line for CVaR
ax.set_xlabel('Portfolio Average Price')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Portfolio Values Across 1000 Scenarios')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Question 3.2: Interpretation (20 points)

**Answer the following questions in 3-5 sentences each:**

1. What is the difference between VaR (Value-at-Risk) and CVaR (Conditional VaR)? Which is more informative for risk management?

2. Based on your histogram, what does the CVaR value mean in practical terms? If you're a portfolio manager, how would you use this information?

3. Why is it important that the scenarios are correlated (i.e., using the same noise vector z for all bonds)? What would happen if we generated independent scenarios for each bond?

**YOUR ANSWER HERE:**

1. [Your explanation of VaR vs CVaR]

2. [Your interpretation of the portfolio risk metrics]

3. [Your explanation of why correlation matters]

---

## Section 4: Correlation & Diversification (25 points)

### Question 4.1: Sector Correlations (10 points)

**Task:**
- Calculate average price by sector across all scenarios
- Compute correlation matrix between sectors
- Create a correlation heatmap
- Identify most/least correlated sector pairs

In [ ]:
# TODO: Calculate sector averages across scenarios
sectors = universe_df['sector'].unique()
sector_scenarios = {}  # sector -> array of 1000 scenario averages

for sector in sectors:
    # TODO: For each sector, calculate average price across all bonds in that sector
    # TODO: Do this for each of the 1000 scenarios
    sector_mask = universe_df['sector'] == sector
    sector_avg = # TODO: Calculate mean across bonds in this sector for each scenario
    sector_scenarios[sector] = sector_avg

# Convert to DataFrame for correlation
sector_df = pd.DataFrame(sector_scenarios)

# TODO: Calculate correlation matrix
sector_corr = # TODO: Calculate correlation matrix of sector_df

# TODO: Create heatmap
fig, ax = plt.subplots(figsize=(10, 8))
# TODO: Use seaborn to create correlation heatmap
plt.title('Sector Correlation Matrix')
plt.tight_layout()
plt.show()

# TODO: Find most/least correlated pairs
# Hint: Extract upper triangle of correlation matrix, find max/min

### Question 4.2: Diversification Analysis (5 points)

**Task:**
- Compare portfolio risk under equal-weight vs single-sector (concentrated) strategy
- Calculate standard deviation for both portfolios

In [ ]:
# Equal-weight portfolio (already calculated above)
equal_weight_std = portfolio_values.std()

# TODO: Pick the largest sector and create concentrated portfolio
largest_sector = # TODO: Find sector with most bonds
sector_mask = universe_df['sector'] == largest_sector
sector_bonds_idx = np.where(sector_mask)[0]

# TODO: Calculate concentrated portfolio values
concentrated_scenarios = # TODO: Extract only bonds in this sector from scenarios
concentrated_values = # TODO: Calculate equal-weighted portfolio within this sector
concentrated_std = # TODO: Calculate standard deviation

print("=" * 60)
print("DIVERSIFICATION COMPARISON")
print("=" * 60)
print(f"Equal-weight portfolio std:  {equal_weight_std:.4f}")
print(f"Concentrated ({largest_sector}) std: {concentrated_std:.4f}")
print(f"Risk reduction from diversification: {(1 - equal_weight_std/concentrated_std)*100:.1f}%")
print("=" * 60)

### Question 4.3: Interpretation (10 points)

**Answer the following questions in 3-5 sentences each:**

1. Which two sectors are most correlated? Least correlated? What might explain these relationships?

2. How much risk reduction do you achieve through diversification (equal-weight vs concentrated)? Is this a meaningful difference?

3. How would you use this correlation information to construct a better diversified portfolio? Be specific about what you would overweight/underweight.

**YOUR ANSWER HERE:**

1. [Your analysis of sector correlations]

2. [Your interpretation of diversification benefits]

3. [Your portfolio construction recommendations]

---

## Final Reflection (Bonus - not graded, but recommended)

**Question**: When would you recommend using a conditional GAN approach vs traditional regression (like ElasticNet) for bond pricing in a real-world setting? Consider factors like:
- Computational cost
- Interpretability
- Risk management needs
- Regulatory requirements

Write 1-2 paragraphs.

**YOUR REFLECTION HERE:**

[Your thoughtful reflection on when to use cGAN vs regression]

---

# Submission Instructions

1. Complete all TODO sections in the code cells
2. Fill in all interpretation sections with your answers
3. Make sure all cells run without errors (Run All)
4. Export notebook as PDF or HTML
5. Submit via Coursera

**Checklist before submission:**
- [ ] All code cells run successfully
- [ ] All TODO comments replaced with working code
- [ ] All interpretation questions answered
- [ ] All plots generated and visible
- [ ] Answers are in your own words (no copy-paste from internet)

---

**Good luck!**